# 机器学习入门

## 工具篇

###### Anaconda & conda
###### Jupyter notebook
###### Markdown
###### LaTeX
###### reveal.js

### Anaconda & conda
###### Anaconda
   建立虚拟环境，管理不同的项目
   
   解决 Python 不同版本之间以及包版本之间的冲突问题
###### conda
   包与环境管理器
   
   类似于pip

###### conda环境管理
conda create -n env_name package_list

conda evn list

activate env_name

deactivte

导出环境包列表
conda env export > environment.yaml

通过环境文件创建环境
conda env create -f environment.yaml

###### conda 常用命令
conda list

conda install numpy pandas scikit-learn

### jupyter notebook
文字表达化编程：直接在代码旁写出叙述性文档

Donald Knuth: 让我们集中精力向人们解释我们希望计算机做什么，而不是指示计算机做什么。

Jupyter notebook 是一种 Web 应用，能让用户将说明文本、数学方程、代码和可视化内容全部组合到一个易于共享的文档中

![title](jupyter notebook arch.jpg)

### Markdown

用 reStructuredText 写文件的工程师鄙视用 Markdown 写文件的工程师，

用 Markdown 写文件的工程师鄙视用 HTML 写文件的工程师，

用 HTML 写文件的工程师鄙视不写文件的工程师，

然后用 LaTeX 写文件的工程师鄙视所有工程师。

# 这是一级标题
## 这是二级标题
### 这是三级标题
*这是斜体字*

_这也是斜体字_

**这是粗体字**

__这也是粗体字__

```
import pandas
print "这是一段代码"
```

### LaTex